In [1]:
import pickle
import time

import pandas as pd
import numpy as np

from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras import optimizers

from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.model_selection import GridSearchCV

Using TensorFlow backend.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([(

In [24]:
def Create_NN_Model(No_Features=100, No_Hidden_Layers=1, No_Hidden_Neurons=30, 
                    Hidden_Activation ="relu", No_OP_Neurons=1, 
                    Output_Activation="sigmoid", Kernel_Initializer="ones",
                    Optimizer="rmsprop", Loss='binary_crossentropy', Metrics =['accuracy']):
  
    classifier = Sequential()

  ## Input Layer
    classifier.add(Dense(No_Hidden_Neurons, activation=Hidden_Activation, 
                       kernel_initializer=Kernel_Initializer, input_dim=No_Features))
  
  ## Hidden layers
    for i in range(No_Hidden_Layers):
        classifier.add(Dense(No_Hidden_Neurons, activation=Hidden_Activation, 
                         kernel_initializer=Kernel_Initializer))
    
  ## Output Layer
    classifier.add(Dense(No_OP_Neurons, activation=Output_Activation, 
                       kernel_initializer=Kernel_Initializer))
  
    classifier.compile(optimizer =Optimizer, loss=Loss, metrics = Metrics)

    return classifier

In [26]:
def Train_NN(NN_classifier, train_data, feature_list=[], Label_Col="Label", Batch_Size=64, Epochs=10):

    train_data.dropna()
    train_data = pd.DataFrame(np.nan_to_num(np.array(train_data)), columns = train_data.columns)
    train_data[Label_Col] = pd.to_numeric(train_data[Label_Col], errors='coerce')
    train_data = train_data.dropna(subset=[Label_Col])
  
    train_features = train_data[feature_list]    
    train_labels = train_data[Label_Col]
    train_labels = train_labels.astype('int')

    NN_classifier.fit(train_features,train_labels, batch_size=Batch_Size, epochs=Epochs)

    eval_model=NN_classifier.evaluate(train_features, train_labels)
    print("Loss: ", eval_model[0])
    print("Accuracy of the model: ", eval_model[1])

    return NN_classifier

In [4]:
## Store trained model in a file to reuse in other codes without training again on same data

def Store_Trained_NN(NN_obj, Filepath):
  
    with open(Filepath, "wb") as file:
        pickle.dump(NN_obj, file)

In [5]:
## Load stored trained model and returns random forest model object

def Load_Trained_NN(Filepath):
  
    with open(Filepath, "rb") as file:
        NN_obj = pickle.load(file)

    return NN_obj

In [6]:
def Evaluate_NN(test_data, NN_Model_FilePath, feature_list=[], Label_Col="label", threshold=0.5):
  
    test_data.dropna()
    test_data = pd.DataFrame(np.nan_to_num(np.array(test_data)),  columns = test_data.columns)
    test_data[Label_Col] = pd.to_numeric(test_data[Label_Col], errors='coerce')
    test_data = test_data.dropna(subset=[Label_Col])

    test_features = test_data[feature_list]
    test_labels = test_data[Label_Col]
    test_labels = test_labels.astype('int')

    NN_obj = Load_Trained_NN(NN_Model_FilePath) 
    predictions = NN_obj.predict(test_features)
    predictions_list = [int(p[0]) for p in predictions]
      
    true_subjective = 0
    true_objective = 0
    false_subjective = 0
    false_objective = 0
    
    for i in range(len(predictions_list)):
        if predictions_list[i] >= threshold:
            predictions_list[i] = 1
            if test_labels[i] == 1:
                true_subjective += 1
            else:
                false_subjective += 1
        else:
            predictions_list[i] = 0
            if test_labels[i] == 0:
                true_objective += 1
            else:
                false_objective += 1
      
    errors = abs(predictions_list - test_labels)

  # Calculate mean absolute error (MAE)
    MAE = round(np.mean(errors), 2)
  
  ## Confusion Matrix and Classification Report
    Confusion_Matrix = confusion_matrix(test_labels,predictions_list)
    Report = classification_report(test_labels,predictions_list)
    
    print("True Subjective : ", true_subjective)
    print("True Objective : ", true_objective)
    print("False Subjective : ", false_subjective)
    print("False Objective : ", false_objective)
    
    print("Accuracy: ", (true_subjective+true_objective)/(true_subjective+true_objective+false_subjective+false_objective)*100)
  
    return MAE, Confusion_Matrix, Report

In [7]:
Column_List = ["embedding"]
Label_Col = "label"
Vector_Size = 100
Embedding_Cols = ["emb"+str(i) for i in range(Vector_Size)]
# Data_Columns = Embedding_Cols
# Data_Columns = Data_Columns.append(Label_Col)
Column_List.append(Label_Col)

Folder_Path = "/Users/pranjali/Downloads/Wiki_BiasDetection/Data/Task1_FinalData/"

Train_Embedding_FilePath = Folder_Path + "Embeddings/train_emb_full.csv"
Test_Embedding_FilePath = Folder_Path + "Embeddings/test_emb_full.csv"
NN_Model_FilePath = "/Users/pranjali/Downloads/Wiki_BiasDetection/Baseline_Models/NN/NN_Train_Model_Full.pkl"

train_data_raw = pd.read_csv(Train_Embedding_FilePath, usecols=Column_List)
test_data_raw = pd.read_csv(Test_Embedding_FilePath, usecols=Column_List)

In [8]:
# Train_Embedding_FilePath2 = Folder_Path + "Embeddings/train_emb.csv"
# Test_Embedding_FilePath2 = Folder_Path + "Embeddings/test_emb.csv"

# train_data_raw_small= pd.read_csv(Train_Embedding_FilePath2, usecols=Column_List)
# test_data_raw_small = pd.read_csv(Test_Embedding_FilePath2, usecols=Column_List)

# # common_train = train_data_raw.merge(train_data_raw_small, on=['embedding','label'])
# # common_test = test_data_raw.merge(test_data_raw_small, on=['embedding','label'])

# train_data_raw = train_data_raw[~train_data_raw.isin(train_data_raw_small)].dropna()
# train_data_raw = train_data_raw[~train_data_raw.isin(test_data_raw_small)].dropna()

# test_data_raw = test_data_raw[~test_data_raw.isin(train_data_raw_small)].dropna()
# test_data_raw = test_data_raw[~test_data_raw.isin(test_data_raw_small)].dropna()

In [9]:
# print(train_data_raw.shape)
# print(test_data_raw.shape)

In [10]:
# train_subjective_data = train_data_raw[train_data_raw["label"]==1]
# train_objective_data = train_data_raw[train_data_raw["label"]==0]

# test_subjective_data = test_data_raw[test_data_raw["label"]==1]
# test_objective_data = test_data_raw[test_data_raw["label"]==0]

In [11]:
# print(train_subjective_data.shape)
# print(train_objective_data.shape)

# print(test_subjective_data.shape)
# print(test_objective_data.shape)

In [12]:
# train_subjective_data_half = pd.DataFrame(train_subjective_data.iloc[:65000,:])
# test_subjective_data_half = pd.DataFrame(test_subjective_data.iloc[:15000,:])

# print(train_subjective_data_half.shape)
# print(test_subjective_data_half.shape)

# train_data_raw = train_data_raw[~train_data_raw.isin(train_subjective_data_half)].dropna()
# test_data_raw = test_data_raw[~test_data_raw.isin(test_subjective_data_half)].dropna()

# print(train_data_raw.shape)
# print(test_data_raw.shape)

In [13]:
# print(train_data_raw.shape)
# print(test_data_raw.shape)

# train_subjective_data = train_data_raw[train_data_raw["label"]==1]
# train_objective_data = train_data_raw[train_data_raw["label"]==0]

# test_subjective_data = test_data_raw[test_data_raw["label"]==1]
# test_objective_data = test_data_raw[test_data_raw["label"]==0]

# print(train_subjective_data.shape)
# print(train_objective_data.shape)

# print(test_subjective_data.shape)
# print(test_objective_data.shape)

In [14]:
def Get_Embeddings(data):
    
    Col_List = ["emb"+str(i) for i in range(100)]
    Col_List.append("label")
    
    Embeddings = []
    
    for i in range(data.shape[0]):
        row = data.iloc[i]
        embedding_str = row["embedding"][1:-1]
        embedding_list = embedding_str.split(',')
        embedding = [float(s) for s in embedding_list]
        embedding.append(int(row["label"]))
        Embeddings.append(embedding)
        
    return pd.DataFrame(Embeddings, columns=Col_List)

train_data = Get_Embeddings(train_data_raw)
test_data = Get_Embeddings(test_data_raw)

In [15]:
print("train_data shape: ", train_data.shape)
print("test_data shape: ", test_data.shape)

train_data shape:  (234311, 101)
test_data shape:  (58579, 101)


In [29]:
train_data = train_data.sample(frac=1)

In [30]:
## Training Phase
start_time = time.time()

NN_Classifier = Create_NN_Model()
NN_obj = Train_NN(NN_Classifier, train_data, Embedding_Cols, Label_Col)
Store_Trained_NN(NN_obj, NN_Model_FilePath)

end_time = time.time()
print("Time required for training: ", end_time - start_time )

Epoch 1/10
234311/234311 [==============================] - 5s 20us/step - loss: 0.6873 - accuracy: 0.6419
Epoch 2/10
234311/234311 [==============================] - 5s 19us/step - loss: 0.6598 - accuracy: 0.6420
Epoch 3/10
234311/234311 [==============================] - 5s 20us/step - loss: 0.6582 - accuracy: 0.6420
Epoch 4/10
234311/234311 [==============================] - 5s 20us/step - loss: 0.6600 - accuracy: 0.6420
Epoch 5/10
234311/234311 [==============================] - 5s 20us/step - loss: 0.6586 - accuracy: 0.6420
Epoch 6/10
234311/234311 [==============================] - 5s 20us/step - loss: 0.6575 - accuracy: 0.6420
Epoch 7/10
234311/234311 [==============================] - 5s 20us/step - loss: 0.6584 - accuracy: 0.6420
Epoch 8/10
234311/234311 [==============================] - 5s 20us/step - loss: 0.6561 - accuracy: 0.6420
Epoch 9/10
234311/234311 [==============================] - 5s 20us/step - loss: 0.6554 - accuracy: 0.6420
Epoch 10/10
234311/234311 [==========

In [31]:
start_time = time.time()

MAE, Confusion_Matrix, Report = Evaluate_NN(test_data, NN_Model_FilePath, Embedding_Cols, Label_Col, 0.5)

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

tn, fp, fn, tp = Confusion_Matrix.ravel()
Accuracy = (tn+tp)/(tn + fp + fn + tp)

print("Accuracy: ", Accuracy*100)

end_time = time.time()
print("Time required for testing: ", end_time - start_time )

True Subjective :  51
True Objective :  20966
False Subjective :  5
False Objective :  37557
Accuracy:  35.87804503320303
MEAN ABSOLUTE ERROR:  0.64


============ CONFUSION MATRIX ===============
[[20966     5]
 [37557    51]]


============ CLASSIFICATION REPORT ==============
              precision    recall  f1-score   support

           0       0.36      1.00      0.53     20971
           1       0.91      0.00      0.00     37608

   micro avg       0.36      0.36      0.36     58579
   macro avg       0.63      0.50      0.27     58579
weighted avg       0.71      0.36      0.19     58579

Accuracy:  35.87804503320303
Time required for testing:  2.8053479194641113


In [ ]:
# train_subjective_data = train_data[train_data["label"]==1]
# train_objective_data = train_data[train_data["label"]==0]

# test_subjective_data = test_data[test_data["label"]==1]
# test_objective_data = test_data[test_data["label"]==0]

In [ ]:
# print(train_subjective_data.shape)
# print(train_objective_data.shape)

# print(test_subjective_data.shape)
# print(test_objective_data.shape)

In [33]:
## Cross Validation

train_data.dropna()
train_data = pd.DataFrame(np.nan_to_num(np.array(train_data)), columns = train_data.columns)
train_data[Label_Col] = pd.to_numeric(train_data[Label_Col], errors='coerce')
train_data = train_data.dropna(subset=[Label_Col])

train_features = train_data[Embedding_Cols]    
train_labels = train_data[Label_Col]
train_labels = train_labels.astype('int')

# create the sklearn model for the network
model_CV = KerasClassifier(build_fn=Create_NN_Model, verbose=1)

# we choose the initializers that came at the top in our previous cross-validation!!
# kernel_initializer = ['random_uniform']
# No_Hidden_Layers = [1, 2]
# No_Hidden_Neurons= [10, 30, 50]
# optimizer = ['adam', 'rmsprop']

# batches = [64*x for x in range(1, 3)]
# epochs = [50, 100, 150]

kernel_initializer = ['random_uniform']
No_Hidden_Layers = [1, 2]
No_Hidden_Neurons= [10, 30, 50]
optimizer = ['adam', 'rmsprop']

batches = [64*x for x in range(1, 3)]
epochs = [20, 50]

## We can also try different learning rates for optimizers. 
## For this create different objects of optimizers with different learning rates and pass list of objects

# grid search for initializer, batch size and number of epochs
param_grid = dict(epochs=epochs, batch_size=batches, Kernel_Initializer=kernel_initializer, 
                 No_Hidden_Layers=No_Hidden_Layers, Optimizer=optimizer)
grid = GridSearchCV(estimator=model_CV, param_grid=param_grid, cv=3, n_jobs=4, refit=True, verbose=2)
grid_result = grid.fit(train_features, train_labels)

Fitting 3 folds for each of 16 candidates, totalling 48 fits


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed: 31.9min
[Parallel(n_jobs=4)]: Done  48 out of  48 | elapsed: 49.8min finished


Epoch 1/50
234311/234311 [==============================] - 5s 23us/step - loss: 0.6289 - accuracy: 0.6571
Epoch 2/50
234311/234311 [==============================] - 5s 21us/step - loss: 0.6179 - accuracy: 0.6673
Epoch 3/50
234311/234311 [==============================] - 5s 22us/step - loss: 0.6127 - accuracy: 0.6725
Epoch 4/50
234311/234311 [==============================] - 5s 21us/step - loss: 0.6093 - accuracy: 0.6755
Epoch 5/50
234311/234311 [==============================] - 5s 22us/step - loss: 0.6070 - accuracy: 0.6773
Epoch 6/50
234311/234311 [==============================] - 6s 24us/step - loss: 0.6050 - accuracy: 0.6792
Epoch 7/50
234311/234311 [==============================] - 8s 34us/step - loss: 0.6026 - accuracy: 0.6816
Epoch 8/50
234311/234311 [==============================] - 8s 35us/step - loss: 0.6006 - accuracy: 0.6828
Epoch 9/50
234311/234311 [==============================] - 9s 39us/step - loss: 0.5995 - accuracy: 0.6834
Epoch 10/50
234311/234311 [==========

In [34]:
# print results of cross validation

print(f'Best Accuracy for {grid_result.best_score_} using {grid_result.best_params_}')
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print(f' mean={mean:.4}, std={stdev:.4} using {param}')

Best Accuracy for 0.6936123337904442 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 2, 'Optimizer': 'rmsprop', 'batch_size': 64, 'epochs': 50}
 mean=0.6856, std=0.0006339 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 20}
 mean=0.691, std=0.0002898 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'adam', 'batch_size': 64, 'epochs': 50}
 mean=0.6808, std=0.002816 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'adam', 'batch_size': 128, 'epochs': 20}
 mean=0.6895, std=0.000445 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'adam', 'batch_size': 128, 'epochs': 50}
 mean=0.6866, std=0.002938 using {'Kernel_Initializer': 'random_uniform', 'No_Hidden_Layers': 1, 'Optimizer': 'rmsprop', 'batch_size': 64, 'epochs': 20}
 mean=0.6901, std=0.001189 using {'Kernel_Initializer': 'random_uniform', 'No_Hid

In [35]:
# ## Train again using best parameter values identified by cross validation and store the trained model

# ## Training Phase
# NN_Classifier = Create_NN_Model(No_Hidden_Layers=grid_result.best_params_.No_Hidden_Layers, No_Hidden_Neurons=grid_result.best_params_.No_Hidden_Neurons, Kernel_Initializer=grid_result.best_params_.Kernel_Initializer, Optimizer=grid_result.best_params_.Optimizer)
# NN_obj = Train_NN(NN_Classifier, train_data, Embedding_Cols, Label_Col, Batch_Size=grid_result.best_params_.batch_size, Epochs=grid_result.best_params_.epochs)
# Store_Trained_NN(NN_obj, NN_Model_FilePath)

## Store models trained with best parameters
Store_Trained_NN(grid, NN_Model_FilePath)

## Evaluation of above model on validation data
MAE, Confusion_Matrix, Report = Evaluate_NN(test_data, NN_Model_FilePath, Embedding_Cols, Label_Col, 0.5)

print("MEAN ABSOLUTE ERROR: ", MAE)

print("\n")
print("============ CONFUSION MATRIX ===============")
print(Confusion_Matrix)

print("\n")
print("============ CLASSIFICATION REPORT ==============")
print(Report)

tn, fp, fn, tp = Confusion_Matrix.ravel()
Accuracy = (tn+tp)/(tn + fp + fn + tp)

print("Accuracy: ", Accuracy*100)


58579/58579 [==============================] - 1s 12us/step
True Subjective :  31848
True Objective :  8469
False Subjective :  12502
False Objective :  5760
Accuracy:  68.8250055480633
MEAN ABSOLUTE ERROR:  0.31


============ CONFUSION MATRIX ===============
[[ 8469 12502]
 [ 5760 31848]]


============ CLASSIFICATION REPORT ==============
              precision    recall  f1-score   support

           0       0.60      0.40      0.48     20971
           1       0.72      0.85      0.78     37608

   micro avg       0.69      0.69      0.69     58579
   macro avg       0.66      0.63      0.63     58579
weighted avg       0.67      0.69      0.67     58579

Accuracy:  68.8250055480633
